In [1]:
from magpie import grasp as gt
import rtde_control
import rtde_receive
from magpie.motor_code import Motors
from magpie import ur5 as ur5
import time
import numpy as np
import copy
import magpie
from magpie.gripper import Gripper

In [104]:
import numpy as np
import copy
from PIL import Image
import open3d as o3d
import matplotlib.pyplot as plt
from magpie.perception import pcd
from open3d.web_visualizer import draw

In [4]:
from magpie import realsense_wrapper as real
rsc = real.RealSense()
rsc.initConnection()

In [5]:
from magpie.perception.label_owlvit import LabelOWLViT
path = "google/owlvit-base-patch32"
label_vit = LabelOWLViT(pth=path)

2024-07-02 17:44:02.363079: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-02 17:44:02.363108: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-02 17:44:02.364127: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-02 17:44:02.369257: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-02 17:44:03.122471: W tensorflow/compiler/tf2

In [6]:
import importlib
importlib.reload(gt)
importlib.reload(ur5)
importlib.reload(magpie.gripper)

<module 'magpie.gripper' from '/home/will/MAGPIE/magpie/gripper.py'>

In [105]:
def seg(index):
    p, rgbd_image = rsc.getPCD()
    image = np.array(rgbd_image.color)
    queries = ["a photo of an apple", "a photo of a lemon", "a photo of a lime", "a photo of a pear", "a photo of a onion"]
    queries = ["a photo of an blue cylindrical container", "a photo of a small red cubic block", "a photo of a screwdriver handle" ]
    queries = ["a photo of a black student ID card", "a photo of a small orange fruit", "a photo of a small plastic bag"]
    queries = ["a photo of an avocado", "a photo of a small orange fruit", "a photo of a small plastic bag", "a photo of a paper airplane"]
    queries = ["a photo of a water bottle with a red top"]
    queries = ["a photo of a blue block"]
    # queries = ["a photo of a tail"]
    # queries = ["a photo of a black handle of a pair of scissors"]
    abbrevq = ["apple", "lemon", "lime", "pear", "onion"]
    abbrevq = ["blue", "block", "handle"]
    abbrevq = ["card", "orange", "bag"]
    abbrevq = ["avocado", "orange", "bag", "paper airplane"]
    abbrevq = ["block"]
    label_vit.set_threshold(0.005)
    # bboxes, uboxes = label_vit.label(image, queries, abbrevq, topk=True, plot=True)
    bboxes, uboxes = label_vit.label(image, queries, abbrevq, topk=True, plot=False)
    rgbd_image, cpcd, tmat, pcaFrame = pcd.get_segment(
                                            label_vit.sorted_labeled_boxes_coords, 
                                            # label_vit.boxes, 
                                            index, 
                                            rgbd_image, 
                                            rsc, 
                                            type="box-dbscan", 
                                            #  type="box", 
                                            #  method="quat", 
                                            method="iterative", 
                                            #  display=False,
                                            display=False,
                                            viz_scale=1000)
    tmat, tmat[:3, 3]
    
    return cpcd, tmat

In [132]:
# load arrays in saved_poses/ folder as array of arrays
saved_poses = []
for i in range(3):
    saved_poses.append(np.load(f'saved_poses/pose_{i+1}.npy', allow_pickle=True))

In [116]:
tmats = []

In [133]:
name = "pose_5"
sleepRate = 3

ur = ur5.UR5_Interface()
try:
    ur.start()
    currentPose = ur.getPose()
    print(currentPose)

    merged_pcd = o3d.geometry.PointCloud()
    
    for k, target_pose in enumerate(saved_poses):
        ur.moveL(target_pose)
        time.sleep(sleepRate)

        # Capture point cloud and transformation matrix
        cpcd, tmat = seg(index=1 if k == 4 else 0)
        o3d.io.write_point_cloud(f"saved_pcd/cpcd{k+1}.ply", cpcd)

        # Transformation matrix for gripper length
        tmat_gripper = np.array([[1, 0, 0, 0],
                                 [0, 1, 0, 0],
                                 [0, 0, 1, 0],
                                 [0, 0, 0, 1]])

        # Get the transformation matrix of the end effector in the base frame
        ee_to_base = ur.get_tcp_pose()

        # Apply transformations
        if k==0:
            cpcd.transform(tmat_gripper)  # Adjust for gripper length
        print(ee_to_base, k)
        cpcd.transform(ee_to_base)  # End effector to base frame

        # Save the transformed point cloud
        o3d.io.write_point_cloud(f"saved_pcd/cpcd{k+1}_t.ply", cpcd)

        # Use ICP to align the new point cloud with the merged point cloud
        if k == 0:
            merged_pcd = cpcd
        else:
            threshold = 0.02  # Distance threshold for ICP
            init_transform = np.eye(4)  # Initial guess for the transformation
            reg_p2p = o3d.pipelines.registration.registration_icp(
                cpcd, merged_pcd, threshold, init_transform,
                o3d.pipelines.registration.TransformationEstimationPointToPoint())
            
            # Transform the new point cloud using the result of ICP
            cpcd.transform(reg_p2p.transformation)
            
            # Merge the point clouds
            merged_pcd += cpcd

    ur.stop()
except Exception as e:
    ur.stop()
    raise e

# Save the merged point cloud
o3d.io.write_point_cloud(f"saved_pcd/merged_pcd.ply", merged_pcd)

# Visualize the merged point cloud
o3d.visualization.draw_geometries([merged_pcd])

Succeeded to open the port
Succeeded to change the baudrate
Moving speed of dxl ID: 1 set to 100 
Moving speed of dxl ID: 2 set to 100 
   0.1695   -0.9852    0.02506  -0.3854    
  -0.9119   -0.1471    0.3832   -0.1741    
  -0.3739   -0.0878   -0.9233    0.1651    
   0         0         0         1         

modified indices: [2 1 2]
modified indices: [0 1 2]
z-axis dot product: [0.849]
[[-0.051 -0.857 -0.514 -0.204]
 [-0.998  0.02   0.066 -0.086]
 [-0.047  0.516 -0.855  0.29 ]
 [ 0.     0.     0.     1.   ]] 0
z-axis dot product: [0.907]
[[-0.038 -0.95  -0.311 -0.309]
 [-0.998  0.018  0.067 -0.087]
 [-0.058  0.313 -0.948  0.229]
 [ 0.     0.     0.     1.   ]] 1
modified indices: [2 2 1]
modified indices: [2 0 1]
z-axis dot product: [0.833]
[[-0.034 -0.967 -0.251 -0.362]
 [-0.998  0.017  0.067 -0.087]
 [-0.061  0.252 -0.966  0.193]
 [ 0.     0.     0.     1.   ]] 2


RTDEControlInterface: Could not receive data from robot...
RTDEControlInterface Exception: Operation canceled
RTDEControlInterface: Robot is disconnected, reconnecting...
RTDEControlInterface Exception: Timeout connecting to UR dashboard server.
RTDEReceiveInterface Exception: Operation canceled


Reconnecting...


In [120]:
merged= o3d.geometry.PointCloud()
for i in range(5):
    cpcd = o3d.io.read_point_cloud(f"saved_pcd/cpcd{i+1}_t.ply")
    merged += cpcd

o3d.visualization.draw_geometries([merged])

In [131]:
o3d.visualization.draw_geometries([merged_pcd])

In [2]:
# set matrix
# tmat = np.array([[ 0.91574736, -0.35602909,  0.18614527, -0.02848649],
#        [-0.40073104, -0.77640133,  0.48643151,  0.14332736],
#        [ 0.02866033,  0.52004256,  0.85365937,  0.33750796],
#        [ 0.        ,  0.        ,  0.        ,  1.        ]])

tmat = np.array([[-0.93571753, -0.30185443, -0.18252837, -0.01585103],
        [ 0.14381697, -0.79893656,  0.58396665, -0.06580893],
        [-0.32210151,  0.52017715,  0.79099074,  0.25853016],
        [ 0.        ,  0.        ,  0.        ,  1.        ]])

aperture = 37
# z_offset = G.aperture_to_z(aperture)/1000.0

mmc = copy.deepcopy(tmat[:3, 3])
# grasp_pose = [mmc[1], -mmc[0], mmc[2]]
grasp_pose = mmc
tmat[:3, 3] = [0, 0, 0]
homePose = None
z_offset = 100/1000.0
z_offset -= 0.035
z_offset

0.065

In [3]:
# move gripper to cartesian pos
# OR orient gripper in place
sleepRate = 1.5
# time.sleep(sleepRate)
ur = ur5.UR5_Interface()
try:
    ur.start()
    # homePose = ur.get_tcp_pose()
    homePose = ur.getPose()
    currentPose = ur.getPose()
    desiredPose = np.matmul(np.array(currentPose), tmat)
    print("Desired Pose: ", desiredPose)
    # in-place (or best attempt) re-orientation
    # ur.moveL(desiredPose)
    ## move to cartesian pos
    gt.move_to_L(grasp_pose, ur, z_offset=z_offset)    
    print("Done moving to block")
    time.sleep(sleepRate)
    time.sleep(sleepRate)
    ur.stop()
except Exception as e:
    ur.stop()
    raise(e)

Dashboard client deadline expired


AttributeError: 'NoneType' object has no attribute 'servoStop'

In [ ]:
# move home
ur = ur5.UR5_Interface()
try:
    ur.start()
    # time.sleep(sleepRate)
    # homePose = np.array([[-0.024, -0.998, -0.062, -0.261],
    #                     [-0.999,  0.021,  0.035, -0.162],
    #                     [-0.033,  0.063, -0.997,  0.221],
    #                     [ 0.   ,  0.   ,  0.   ,  1.   ]])
    ur.moveL(homePose)
    # gt_home = ur.getPose()
    # print(np.array(gt_home))
    time.sleep(sleepRate * 3)
    ur.stop()
except Exception as e:
    ur.stop()

In [ ]:
# move gripper to cartesian pos
# OR orient gripper in place
name = "pose_5"
sleepRate = 4
# time.sleep(sleepRate)
ur = ur5.UR5_Interface()
try:
    ur.start()
    currentPose = ur.getPose()
    print(currentPose)

    ur.moveL(saved_poses[4])
    time.sleep(sleepRate)
    time.sleep(sleepRate)
    ur.stop()
except Exception as e:
    ur.stop()
    raise(e)

In [ ]:
index = 1
# this does the [x, y, z] --> [y, -x, z] grasp pose switch, and the -y inversio on the y-axis orientation
# rgbd_image, cpcd, tmat = pcd|.get_segment(label_vit.boxes, index, rgbd_image, rsc, type="box", display=False)
rgbd_image, cpcd, tmat, pcaFrame = pcd.get_segment(
                                        label_vit.sorted_labeled_boxes_coords, 
                                        # label_vit.boxes, 
                                         index, 
                                         rgbd_image, 
                                         rsc, 
                                         type="box-dbscan", 
                                        #  type="box", 
                                        #  method="quat", 
                                         method="iterative", 
                                        #  display=False,
                                         display=True,
                                         viz_scale=1000)
tmat, tmat[:3, 3]



In [ ]:
# DO NOT EXECUTE

In [ ]:
# # orient and move gripper to cartesian pos
# sleepRate = 1.5
# ur = ur5.UR5_Interface()
# try:
#     servoPort = "/dev/ttyACM0"
#     ur.start()
#     time.sleep(sleepRate)
#     homePose = ur.get_tcp_pose()    
#     '''
#     stopping point 2/15:
#     this works so far
#     take home pos
#     add translation on the gripper frame to the goal x, y
#     apply tmat to the goal
#     move to goal (translates on x, y and orients at the same time)
#     ## does this address the fact that orientation is not achievable in standstill?

#     then: take orig Z (from original gripper frame): [0, 0, dZ]
#     transform by tmat (the current orientation of the gripper)
#     add transformed dZ to the goal position
#     move
#     works!
#     '''
#     dX,dY,dZ = gt.get_world_frame(grasp_pose, ur, z_offset=z_offset)

#     goal1 = copy.deepcopy(homePose)
#     goal1 = np.array(goal1)
#     # goal1[:3, 3] += [dX, dY, 0]
#     goal1[:3, 3] += [dX, dY, dZ]
#     goal1 = np.array(goal1) @ tmat
#     ur.moveL(goal1)
#     time.sleep(sleepRate)

#     # todo: take currentPose and validate orientation?
#     goal2 = goal1
#     posd = [0, 0, dZ] @ tmat[:3, :3]
#     goal2 = np.array(goal2)
#     goal2[:3, 3] += posd
#     # ur.moveL(goal2)
#     time.sleep(sleepRate)
#     '''
#     end of working section
#     '''
#     ur.stop()
# except Exception as e:
#     ur.stop()
#     raise(e)